# Exoplanet Target List

A list of exoplanets that are good targets for follow-up observations. 

- Download transiting planets from the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/).
- Resolve the host star brightness and distance using [Simbad](http://simbad.u-strasbg.fr/simbad/).
- Query [TESS](https://tess.mit.edu/) to estimate the total number of light curves available.
- Query [DACE](https://dace.unige.ch/DACE.php) for radial velocity measurements.
- Estimate the transit and eclipse times for each planet propagated forward to 2030.
- Compute equilibrium temperatures and eclipse depths.

In [29]:
import os
import json
from dace_query.spectroscopy import Spectroscopy

class RadialVelocityQuery:

    def __init__(self, target=None):
        self.target = target
        self.data = {
            "target": target,
            "time": [],
            "instrument": [],
            "rv": [],
            "rv_err": []
        }

        self.query_dace()

    def query_dace(self):

        try:
            # query dace for number of RV observations
            observedTargets = Spectroscopy.query_database(
                limit=1000,
                filters={"public": {"is": True}, "obj_id_catname": {"contains": [self.target]}},
                output_format="pandas"
            )
        except:
            print(f"error for {self.target}")

        if len(observedTargets) == 0:
            print(f"No RV observations for {self.target}")
        else:
            print(f"Found {len(observedTargets)} RV observations for {self.target} from DACE")

            # extract time, instrument name, rv, error
            self.data["time"].extend(observedTargets["obj_date_bjd"] + 2450000) # TODO check this conversion
            self.data["instrument"].extend(observedTargets["ins_name"])
            self.data["rv"].extend(observedTargets["spectro_ccf_rv"])
            self.data["rv_err"].extend(observedTargets["spectro_ccf_rv_err"])

            # TODO add activity index? other metrics?

        # def query_database?
        # def load_from_disk?

    def save_to_disk(self, path):
        # save data as json
        with open(path, 'w') as f:
            json.dump(self.data, f)

    def load_from_disk(self, path):
        # load data from json
        with open(path, 'r') as f:
            self.data = json.load(f)
        self.target = self.data["target"]

In [30]:
target = "WASP-77A"

rv = RadialVelocityQuery(target=target)
rv.data

Found 19 RV observations for WASP-77A from DACE


{'target': 'WASP-77A',
 'time': [2505188.6258,
  2505891.5721,
  2505914.6783,
  2505832.8615,
  2505856.7095,
  2505169.692,
  2505832.904,
  2505916.6681,
  2505170.6625,
  2505915.6775,
  2505891.7468,
  2505163.6091,
  2505890.7386,
  2505889.7458,
  2505890.537,
  2505832.911,
  2505069.7803,
  2505917.65,
  2505918.6645],
 'instrument': ['CORALIE',
  'HARPS',
  'CORALIE',
  'HARPS',
  'CORALIE',
  'CORALIE',
  'HARPS',
  'CORALIE',
  'CORALIE',
  'CORALIE',
  'HARPS',
  'CORALIE',
  'HARPS',
  'HARPS',
  'HARPS',
  'HARPS',
  'CORALIE',
  'CORALIE',
  'CORALIE'],
 'rv': [1969.6,
  1767.0,
  1710.5,
  1462.6,
  1865.5,
  1967.3,
  1511.1,
  1708.7,
  1570.6,
  1332.8,
  1988.5,
  1336.2,
  1865.6,
  1406.3,
  2022.0,
  1523.3,
  1324.6,
  1954.4,
  1554.2],
 'rv_err': [5.4,
  4.4,
  4.3,
  2.0,
  5.3,
  4.9,
  2.4,
  4.8,
  5.3,
  4.4,
  4.6,
  5.2,
  4.1,
  3.6,
  3.7,
  2.5,
  4.9,
  4.7,
  6.5]}

In [21]:
data["obj_date_bjd"]

0     55188.6258
1     55891.5721
2     55914.6783
3     55832.8615
4     55856.7095
5     55169.6920
6     55832.9040
7     55916.6681
8     55170.6625
9     55915.6775
10    55891.7468
11    55163.6091
12    55890.7386
13    55889.7458
14    55890.5370
15    55832.9110
16    55069.7803
17    55917.6500
18    55918.6645
Name: obj_date_bjd, dtype: float64

In [4]:
# fill out pl_ratdor (a/Rs)
# fill out pl_ratror (Rp/Rs)
# estimate sa
import astropy.units as u
import astropy.constants as c

rsun = c.R_sun.to(u.m).value
msun = c.M_sun.to(u.kg).value
mjup = c.M_jup.to(u.kg).value
rjup = c.R_jup.to(u.m).value
mearth = c.M_earth.to(u.kg).value
rearth = c.R_earth.to(u.m).value
au = c.au.to(u.m).value
G = c.G.to(u.AU**3/u.M_sun/u.day**2).value # day, AU, Msun

# logg[cm/s2], rs[rsun]
stellar_mass = lambda logg,rs: ((rs*rsun)**2 * 10**logg / c.G.value)/100./msun

# keplerian semi-major axis (au)
sa = lambda m,P : (G*m*P**2/(4*np.pi**2) )**(1./3)

# fill out stellar mass and a/Rs if missing
for i in targets.index:
    if np.isnan(targets.loc[i, 'st_mass']):
        targets.loc[i, 'st_mass'] = stellar_mass(targets.loc[i, 'st_logg'], targets.loc[i, 'st_rad'])

    if np.isnan(targets.loc[i, 'pl_ratdor']):
        targets.loc[i, 'pl_ratdor'] = sa(targets.loc[i, 'st_mass'], targets.loc[i, 'pl_orbper']) / targets.loc[i, 'st_rad']
        targets.loc[i, 'pl_ratdorerr1'] = 0.05 * targets.loc[i, 'pl_ratdor']
        targets.loc[i, 'pl_ratdorerr2'] = -0.05 * targets.loc[i, 'pl_ratdor']

    if np.isnan(targets.loc[i, 'pl_ratror']):
        targets.loc[i, 'pl_ratror'] = (rjup * targets.loc[i, 'pl_radj']) / (rsun * targets.loc[i, 'st_rad'])
        targets.loc[i, 'pl_ratrorerr1'] = 0.05 * targets.loc[i, 'pl_ratror']
        targets.loc[i, 'pl_ratrorerr2'] = -0.05 * targets.loc[i, 'pl_ratror']

In [5]:
# if ecc is nan, set to 0, along with error
targets.pl_orbeccen = targets.pl_orbeccen.fillna(0)
targets.pl_orbeccenerr1 = targets.pl_orbeccenerr1.fillna(0)
targets.pl_orbeccenerr2 = targets.pl_orbeccenerr2.fillna(0)

# if pl_orblper is nan, set to 0, along with error
targets.pl_orblper = targets.pl_orblper.fillna(0)
targets.pl_orblpererr1 = targets.pl_orblpererr1.fillna(0)
targets.pl_orblpererr2 = targets.pl_orblpererr2.fillna(0)

# if pl_orbincl is nan, set to 90, along with error
targets.pl_orbincl = targets.pl_orbincl.fillna(90)
targets.pl_orbinclerr1 = targets.pl_orbinclerr1.fillna(0)
targets.pl_orbinclerr2 = targets.pl_orbinclerr2.fillna(0)

targets

,pl_name,hostname,pl_radj,pl_radjerr1,ra,dec,pl_ratdor,pl_ratdorerr1,pl_ratdorerr2,pl_orbincl,...,st_teff,st_tefferr1,st_tefferr2,st_met,st_logg,st_loggerr1,st_loggerr2,st_mass,st_rad,st_raderr1
0,K2-283 b,K2-283,0.314,0.014,13.194368,9.692918,8.230000,0.560000,-1.390000,90.000,...,5060.0,70.0,-70.0,0.280,4.56,0.02,-0.02,0.89,0.82,0.01
1,TOI-1260 c,TOI-1260,0.246,0.006,157.144071,65.854199,20.990000,0.630000,-0.630000,87.970,...,4227.0,85.0,-85.0,-0.100,4.57,0.05,-0.05,0.68,0.67,0.01
2,HD 28109 c,HD 28109,0.377,0.010,65.238306,-68.102688,0.213398,0.010670,-0.010670,89.543,...,6120.0,50.0,-50.0,0.000,4.13,0.10,-0.10,1.26,1.45,0.04
3,TOI-238 b,TOI-238,0.125,0.008,349.231081,-18.606646,0.029111,0.001456,-0.001456,84.600,...,5059.0,89.0,-89.0,-0.114,4.58,0.02,-0.02,0.79,0.73,0.02
4,TOI-1470 b,TOI-1470,0.194,0.004,10.089143,61.213387,13.050000,0.140000,-0.150000,87.420,...,3709.0,11.0,-11.0,0.000,4.77,0.02,-0.02,0.47,0.47,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,HD 183579 b,HD 183579,0.317,0.013,293.286615,-54.532728,29.100000,0.800000,-1.100000,89.330,...,5788.0,44.0,-44.0,-0.023,4.50,0.03,-0.03,1.03,0.99,0.04
1451,TOI-712 b,TOI-712,0.183,0.001,92.936106,-65.825839,0.118233,0.005912,-0.005912,88.220,...,4622.0,61.0,-59.0,-0.020,4.65,0.02,-0.02,0.73,0.67,0.02
1452,TOI-1749 c,TOI-1749,0.189,0.011,282.737228,64.419466,0.080798,0.004040,-0.004040,88.800,...,3985.0,55.0,-55.0,-0.260,4.70,0.05,-0.05,0.58,0.55,0.03
1453,TOI-2443 b,TOI-2443,0.247,0.004,40.179861,1.199676,26.293000,0.952000,-2.053000,89.380,...,4357.0,100.0,-100.0,-0.439,4.71,0.10,-0.10,0.66,0.73,0.07


# Query for host star brightness and TIC ID

In [6]:
from astroquery.mast import Catalogs

# Query SIMBAD for V-mag and TIC ID
targets['Jmag'] = np.nan
targets['Tmag'] = np.nan
targets['TIC'] = np.nan

for target in tqdm(targets.pl_name):
    time.sleep(1) # avoid rate limits

    # Query the TESS catalog
    try:
        catalog_data = Catalogs.query_object(target, catalog="TIC")
    except:
        # use RA and dec to query
        ra = targets.loc[targets.pl_name == target, 'ra'].values[0]
        dec = targets.loc[targets.pl_name == target, 'dec'].values[0]
        catalog_data = Catalogs.query_object(f"{ra} {dec}", catalog="TIC")

    # add to table
    targets.loc[targets.pl_name == target, 'Jmag'] =  catalog_data['Jmag'][0]
    targets.loc[targets.pl_name == target, 'Tmag'] =  catalog_data['Tmag'][0]
    targets.loc[targets.pl_name == target, 'TIC'] =  catalog_data['ID'][0]

  1%|          | 8/1451 [00:24<1:08:50,  2.86s/it]

In [ ]:
# filter targets brighter than 13 mag
targets = targets[targets.Tmag < 13]

# Estimate mid-transit and mid-eclipse ephemeris uncertainties

In [ ]:
# time in julian days of 2030
t2030 = Time('2030-01-01T00:00:00.000', format='isot', scale='utc').jd

# forward propagated to 2030
targets['tmid'] = np.nan
targets['tmid_err'] = np.nan
targets['emid'] = np.nan
targets['emid_err'] = np.nan
targets['per'] = np.nan
targets['per_err'] = np.nan
targets['ars'] = np.nan
targets['ars_err'] = np.nan
targets['inc'] = np.nan
targets['inc_err'] = np.nan
targets['ecc'] = np.nan
targets['ecc_err'] = np.nan
targets['omega'] = np.nan
targets['omega_err'] = np.nan
targets['Teq'] = np.nan
targets['Teq_err'] = np.nan
targets['edepth_vis'] = np.nan
targets['edepth_vis_err'] = np.nan
targets['edepth_ir'] = np.nan
targets['edepth_ir_err'] = np.nan
targets['depth'] = np.nan
targets['depth_err'] = np.nan


for target in tqdm(targets.pl_name):

    # mid-transit
    tmid = targets.loc[targets.pl_name == target, 'pl_tranmid'].values[0]
    tmid_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_tranmiderr1'].values[0]),
                        np.abs(targets.loc[targets.pl_name == target, 'pl_tranmiderr2'].values[0])])
    if np.isnan(tmid_err):
        tmid_err = 1e-5
    # update df
    targets.loc[targets.pl_name == target, 'tmid'] = tmid
    targets.loc[targets.pl_name == target, 'tmid_err'] = tmid_err

    # orbital period
    period = targets.loc[targets.pl_name == target, 'pl_orbper'].values[0]
    period_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_orbpererr1'].values[0]),
                          np.abs(targets.loc[targets.pl_name == target, 'pl_orbpererr2'].values[0])])
    if np.isnan(period_err):
        period_err = 1e-6
    targets.loc[targets.pl_name == target, 'per'] = period
    targets.loc[targets.pl_name == target, 'per_err'] = period_err    

    # semi-major axis scaled by Rs
    ars = targets.loc[targets.pl_name == target, 'pl_ratdor'].values[0]
    ars_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_ratdorerr1'].values[0]),
                       np.abs(targets.loc[targets.pl_name == target, 'pl_ratdorerr2'].values[0])])
    if np.isnan(ars_err):
        ars_err = 0.1
    targets.loc[targets.pl_name == target, 'ars'] = ars
    targets.loc[targets.pl_name == target, 'ars_err'] = ars_err

    # inclination
    inc = targets.loc[targets.pl_name == target, 'pl_orbincl'].values[0]
    inc_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_orbinclerr1'].values[0]),
                       np.abs(targets.loc[targets.pl_name == target, 'pl_orbinclerr2'].values[0])])
    if np.isnan(inc_err):
        inc_err = 1
    targets.loc[targets.pl_name == target, 'inc'] = inc
    targets.loc[targets.pl_name == target, 'inc_err'] = inc_err
    
    # eccentricity
    ecc = targets.loc[targets.pl_name == target, 'pl_orbeccen'].values[0]
    ecc_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_orbeccenerr1'].values[0]),
                       np.abs(targets.loc[targets.pl_name == target, 'pl_orbeccenerr2'].values[0])])
    if np.isnan(ecc_err):
        ecc_err = 0.01
    targets.loc[targets.pl_name == target, 'ecc'] = ecc
    targets.loc[targets.pl_name == target, 'ecc_err'] = ecc_err
    
    # longitude of periastron
    omega = targets.loc[targets.pl_name == target, 'pl_orblper'].values[0]
    omega_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_orblpererr1'].values[0]),
                       np.abs(targets.loc[targets.pl_name == target, 'pl_orblpererr2'].values[0])])
    if np.isnan(omega_err):
        omega_err = 1
    targets.loc[targets.pl_name == target, 'omega'] = omega
    targets.loc[targets.pl_name == target, 'omega_err'] = omega_err
    
    # stellar temperature
    Teff = targets.loc[targets.pl_name == target, 'st_teff'].values[0]
    Teff_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'st_tefferr1'].values[0]),
                          np.abs(targets.loc[targets.pl_name == target, 'st_tefferr2'].values[0])])
    if np.isnan(Teff_err):
        Teff_err = 100

    # rprs
    rprs = targets.loc[targets.pl_name == target, 'pl_ratror'].values[0]
    rprs_err = np.nanmean([np.abs(targets.loc[targets.pl_name == target, 'pl_ratrorerr1'].values[0]),
                            np.abs(targets.loc[targets.pl_name == target, 'pl_ratrorerr2'].values[0])])
    if np.isnan(rprs_err):
        rprs_err = 0.001

    # find number of epochs till first transit of 2030
    n_epochs = np.ceil((t2030 - tmid) / period)

    # propagate epehermis to 2030 with monte carlo error estimation using 10000 samples
    tmids = []
    emids = []
    teqs = []
    edepth_vis = []
    edepth_ir = []
    depths = []

    for i in range(1000):
        # randomize parameters
        rtmid = np.random.normal(tmid, tmid_err)
        rperiod = np.random.normal(period, period_err)
        rars = np.random.normal(ars, ars_err)
        rinc = np.random.normal(inc, inc_err)
        recc = np.random.normal(ecc, ecc_err)
        romega = np.random.normal(omega, omega_err)
        rTeff = np.random.normal(Teff, Teff_err)
        rrprs = np.random.normal(rprs, rprs_err)
    
        # propagate to 2030
        tmids.append( rtmid + n_epochs * rperiod )

        try:
            emid = eclipse_mid_time(rperiod, rars, recc, rinc, romega, rtmid)
        except:
            emid = np.nan
    
        emids.append( emid + n_epochs * rperiod )

        # equilibrium temperature assuming albedo = 0.1 and uniform heat distribution
        rTe = rTeff * (1 - 0.1)**0.25 * np.sqrt(0.5/rars)
        teqs.append(rTe)

        # eclipse depth
        edepth_vis.append(eclipse_depth(rTeff, rTe, rrprs**2, 0.5, 1.0)) # TESS bandpass
        edepth_ir.append(eclipse_depth(rTeff, rTe, rrprs**2, 3.0, 5.0)) # JWST bandpass

        # transit depth
        depths.append(rrprs**2)

    # update df
    targets.loc[targets.pl_name == target, 'tmid'] = np.mean(tmids)
    targets.loc[targets.pl_name == target, 'tmid_err'] = np.std(tmids)
    targets.loc[targets.pl_name == target, 'emid'] = np.mean(emids)
    targets.loc[targets.pl_name == target, 'emid_err'] = np.std(emids)
    targets.loc[targets.pl_name == target, 'Teq'] = np.mean(teqs)
    targets.loc[targets.pl_name == target, 'Teq_err'] = np.std(teqs)
    targets.loc[targets.pl_name == target, 'edepth_vis'] = np.mean(edepth_vis)
    targets.loc[targets.pl_name == target, 'edepth_vis_err'] = np.std(edepth_vis)
    targets.loc[targets.pl_name == target, 'edepth_ir'] = np.mean(edepth_ir)
    targets.loc[targets.pl_name == target, 'edepth_ir_err'] = np.std(edepth_ir)
    targets.loc[targets.pl_name == target, 'depth'] = np.mean(depths)
    targets.loc[targets.pl_name == target, 'depth_err'] = np.std(depths)


/Users/kpearson/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/var/folders/5c/hh52mnk94s3fyqw76f2dl_kr0000gq/T/ipykernel_73161/3430795043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['tmid'] = np.nan
/var/folders/5c/hh52mnk94s3fyqw76f2dl_kr0000gq/T/ipykernel_73161/3430795043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['tmid

# Query TESS for number of light curves

In [ ]:
for target in tqdm(targets.pl_name):
    time.sleep(1)

    # search lightkurve for target
    try:
        search_result = lk.search_lightcurve(target, mission='TESS')
    except:
        continue

    if len(search_result) == 0:
        continue

    sectors = np.unique(search_result.mission)

    # add number of sectors and number of lightcurves
    targets.loc[targets.pl_name == target, 'n_sectors'] = len(sectors)
    targets.loc[targets.pl_name == target, 'n_lc'] = int(len(sectors) * 27.4 / targets.loc[targets.pl_name == target, 'pl_orbper'])

  0%|          | 0/1159 [00:00<?, ?it/s]/var/folders/5c/hh52mnk94s3fyqw76f2dl_kr0000gq/T/ipykernel_73161/3138566372.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.loc[targets.pl_name == target, 'n_sectors'] = len(sectors)
/var/folders/5c/hh52mnk94s3fyqw76f2dl_kr0000gq/T/ipykernel_73161/3138566372.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.loc[targets.pl_name == target, 'n_lc'] = int(len(sectors) * 27.4 / targets.loc[targets.pl_name == target, 'pl_orbper'])
100%|██████████| 11

# Query DACE for RV measurements


In [ ]:
targets['n_rv'] = np.nan

columns_of_interest = ['ins_name', 'ins_mode', 'obj_date_bjd', 'spectro_ccf_rv', 'spectro_ccf_rv_err', 'spectro_analysis_rhk', 'spectro_analysis_rhk_err']

os.makedirs('radial_velocity/data', exist_ok=True)

for target in tqdm(targets.hostname):
    time.sleep(1) # avoid rate limits

    try:
        # query dace for number of RV observations
        observedTargets = Spectroscopy.query_database(
            limit=1000,
            filters={"public": {"is": True}, "obj_id_catname": {"contains": [target]}},
            output_format="pandas"
        )
    except:
        print(f"error for {target}")
        continue

    if len(observedTargets) == 0:
        continue

    # add to table
    targets.loc[targets.hostname == target, 'n_rv'] = len(observedTargets)

    # save rv measurements to disk
    observedTargets = observedTargets[columns_of_interest]
    observedTargets.to_csv(f'radial_velocity/data/{target}.csv', index=False)


/var/folders/5c/hh52mnk94s3fyqw76f2dl_kr0000gq/T/ipykernel_73161/4247552753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['n_rv'] = np.nan
100%|██████████| 1159/1159 [34:35<00:00,  1.79s/it]


In [ ]:
# conver data frame to a sortable table in html
targets.to_html('targets.html', index=False, classes=['sortable'], render_links=True)

# save table to disk
targets.to_csv('targets.csv', index=False)

# show table in notebook
#from IPython.display import HTML
#HTML(targets.to_html(index=False, classes=['sortable'], render_links=True))

In [ ]:
# clean up some columns and names so we can make the table more presentable
columns_to_remove = ['pl_ratdor', 'pl_ratdorerr1', 'pl_ratdorerr2', 'pl_orbincl',
       'pl_orbinclerr1', 'pl_orbinclerr2', 'pl_orbper', 'pl_orbpererr1',
       'pl_orbpererr2', 'pl_orbeccen', 'pl_orbsmax', 'pl_orbsmaxerr1',
       'pl_orbsmaxerr2', 'pl_orblper', 'pl_tranmid', 'pl_tranmiderr1',
       'pl_tranmiderr2', 'pl_orbeccenerr1', 'pl_orbeccenerr2',
       'pl_orblpererr1', 'pl_orblpererr2', 'pl_ratrorerr2', 
       'st_tefferr2', 'st_loggerr2']

subtargets = targets.drop(columns=columns_to_remove)

# save to disk
subtargets.to_csv('subtargets.csv', index=False)

subtargets

,pl_name,hostname,pl_radj,pl_radjerr1,ra,dec,pl_ratror,pl_ratrorerr1,st_teff,st_tefferr1,...,Teq_err,edepth_vis,edepth_vis_err,edepth_ir,edepth_ir_err,depth,depth_err,n_sectors,n_lc,n_rv
1,TOI-1260 c,TOI-1260,0.246,0.006,157.144071,65.854199,0.03770,0.00070,4227.0,85.0,...,16.190758,8.896135e-13,4.903142e-13,0.000006,7.816394e-07,0.001424,0.000052,6.0,21.0,NaN
2,HD 28109 c,HD 28109,0.377,0.010,65.238306,-68.102688,0.02632,0.00027,6120.0,50.0,...,241.608543,2.218284e-03,1.495257e-04,0.001158,4.352508e-05,0.000693,0.000014,34.0,16.0,NaN
3,TOI-238 b,TOI-238,0.125,0.008,349.231081,-18.606646,0.01721,0.00083,5059.0,89.0,...,627.018541,9.374696e-03,1.058193e-03,0.001632,1.661627e-04,0.000297,0.000029,NaN,NaN,NaN
4,TOI-1470 b,TOI-1470,0.194,0.004,10.089143,61.213387,0.04260,0.00080,3709.0,11.0,...,4.300314,2.107817e-11,2.756304e-12,0.000018,8.084880e-07,0.001816,0.000064,4.0,43.0,NaN
5,V1298 Tau b,V1298 Tau,0.916,0.052,61.331654,20.157032,0.07000,0.00230,5050.0,100.0,...,19.250610,5.108344e-12,3.331914e-12,0.000023,3.407404e-06,0.004910,0.000327,2.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,HD 183579 b,HD 183579,0.317,0.013,293.286615,-54.532728,0.03300,0.00063,5788.0,44.0,...,13.404035,5.242015e-12,2.006356e-12,0.000007,6.302274e-07,0.001091,0.000039,3.0,4.0,NaN
1448,TOI-712 b,TOI-712,0.183,0.001,92.936106,-65.825839,0.02772,0.00180,4622.0,61.0,...,257.760808,7.320957e-03,8.414798e-04,0.001941,1.872328e-04,0.000769,0.000069,25.0,71.0,NaN
1449,TOI-1749 c,TOI-1749,0.189,0.011,282.737228,64.419466,0.03530,0.00080,3985.0,55.0,...,272.715551,2.529917e-02,2.017525e-03,0.004139,2.187889e-04,0.001249,0.000054,28.0,170.0,NaN
1450,TOI-2443 b,TOI-2443,0.247,0.004,40.179861,1.199676,0.03470,0.00060,4357.0,100.0,...,23.028333,1.184708e-13,1.406784e-13,0.000003,7.222095e-07,0.001204,0.000041,NaN,NaN,NaN


In [ ]:
# paragraph about TTVs (multiplanet justification)
subtargets.keys()

Index(['pl_name', 'hostname', 'pl_radj', 'pl_radjerr1', 'ra', 'dec',
       'pl_ratror', 'pl_ratrorerr1', 'st_teff', 'st_tefferr1', 'st_met',
       'st_logg', 'st_loggerr1', 'st_mass', 'st_rad', 'st_raderr1', 'Jmag',
       'Tmag', 'TIC', 'tmid', 'tmid_err', 'emid', 'emid_err', 'per', 'per_err',
       'ars', 'ars_err', 'inc', 'inc_err', 'ecc', 'ecc_err', 'omega',
       'omega_err', 'Teq', 'Teq_err', 'edepth_vis', 'edepth_vis_err',
       'edepth_ir', 'edepth_ir_err', 'depth', 'depth_err', 'n_sectors', 'n_lc',
       'n_rv'],
      dtype='object')

In [ ]:
# reset index
subtargets = subtargets.reset_index(drop=True)
subtargets

,pl_name,hostname,pl_radj,pl_radjerr1,ra,dec,pl_ratror,pl_ratrorerr1,st_teff,st_tefferr1,...,Teq_err,edepth_vis,edepth_vis_err,edepth_ir,edepth_ir_err,depth,depth_err,n_sectors,n_lc,n_rv
0,TOI-1260 c,TOI-1260,0.246,0.006,157.144071,65.854199,0.03770,0.00070,4227.0,85.0,...,16.190758,8.896135e-13,4.903142e-13,0.000006,7.816394e-07,0.001424,0.000052,6.0,21.0,NaN
1,HD 28109 c,HD 28109,0.377,0.010,65.238306,-68.102688,0.02632,0.00027,6120.0,50.0,...,241.608543,2.218284e-03,1.495257e-04,0.001158,4.352508e-05,0.000693,0.000014,34.0,16.0,NaN
2,TOI-238 b,TOI-238,0.125,0.008,349.231081,-18.606646,0.01721,0.00083,5059.0,89.0,...,627.018541,9.374696e-03,1.058193e-03,0.001632,1.661627e-04,0.000297,0.000029,NaN,NaN,NaN
3,TOI-1470 b,TOI-1470,0.194,0.004,10.089143,61.213387,0.04260,0.00080,3709.0,11.0,...,4.300314,2.107817e-11,2.756304e-12,0.000018,8.084880e-07,0.001816,0.000064,4.0,43.0,NaN
4,V1298 Tau b,V1298 Tau,0.916,0.052,61.331654,20.157032,0.07000,0.00230,5050.0,100.0,...,19.250610,5.108344e-12,3.331914e-12,0.000023,3.407404e-06,0.004910,0.000327,2.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,HD 183579 b,HD 183579,0.317,0.013,293.286615,-54.532728,0.03300,0.00063,5788.0,44.0,...,13.404035,5.242015e-12,2.006356e-12,0.000007,6.302274e-07,0.001091,0.000039,3.0,4.0,NaN
1155,TOI-712 b,TOI-712,0.183,0.001,92.936106,-65.825839,0.02772,0.00180,4622.0,61.0,...,257.760808,7.320957e-03,8.414798e-04,0.001941,1.872328e-04,0.000769,0.000069,25.0,71.0,NaN
1156,TOI-1749 c,TOI-1749,0.189,0.011,282.737228,64.419466,0.03530,0.00080,3985.0,55.0,...,272.715551,2.529917e-02,2.017525e-03,0.004139,2.187889e-04,0.001249,0.000054,28.0,170.0,NaN
1157,TOI-2443 b,TOI-2443,0.247,0.004,40.179861,1.199676,0.03470,0.00060,4357.0,100.0,...,23.028333,1.184708e-13,1.406784e-13,0.000003,7.222095e-07,0.001204,0.000041,NaN,NaN,NaN


In [ ]:
# count targets with positive n_RV
n_rv = len(subtargets[subtargets.n_rv > 0])
print(f"Number of targets with RV measurements: {n_rv}")

# mask for targets with RV measurements and TESS lightcurves
mask = (subtargets.n_rv > 0) & (subtargets.n_lc > 0)

# count targets with positive n_RV and n_lc
n_rv_lc = len(subtargets[mask])
print(f"Number of targets with RV measurements and TESS lightcurves: {n_rv_lc}")

Number of targets with RV measurements: 129
Number of targets with RV measurements and TESS lightcurves: 118


In [ ]:
candidates = subtargets[mask]

# sort by largerst emid error
candidates = candidates.sort_values('emid_err', ascending=False)

candidates

,pl_name,hostname,pl_radj,pl_radjerr1,ra,dec,pl_ratror,pl_ratrorerr1,st_teff,st_tefferr1,...,Teq_err,edepth_vis,edepth_vis_err,edepth_ir,edepth_ir_err,depth,depth_err,n_sectors,n_lc,n_rv
715,K2-13 b,K2-13,0.169,0.085,170.155476,2.502753,0.02260,0.00130,5698.0,45.0,...,NaN,NaN,NaN,NaN,NaN,0.000513,0.000058,3.0,2.0,11.0
1022,K2-3 d,K2-3,0.130,0.005,172.335371,-1.455136,0.02449,0.00041,3844.0,61.0,...,6.274876,2.498832e-24,3.201019e-24,8.353022e-09,1.752982e-09,0.000600,0.000020,3.0,1.0,80.0
413,TOI-201 b,TOI-201,1.008,0.012,87.401782,-54.910423,0.07890,0.00130,6394.0,75.0,...,19.410738,4.862969e-13,4.167352e-13,1.353390e-05,2.379716e-06,0.006227,0.000207,32.0,16.0,11.0
489,TOI-125 d,TOI-125,0.261,0.015,23.593425,-66.676350,0.03170,0.00180,5320.0,39.0,...,7.917452,1.351942e-13,4.273402e-14,2.883490e-06,3.739137e-07,0.001014,0.000114,5.0,6.0,120.0
51,K2-132 b,K2-132,1.300,0.070,182.166267,-8.747217,0.03250,0.00140,4840.0,90.0,...,87.508879,5.520236e-07,3.625229e-07,1.168405e-04,1.942974e-05,0.001057,0.000082,2.0,5.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,WASP-12 b,WASP-12,1.937,0.056,97.636645,29.672266,0.11700,0.00020,6360.0,140.0,...,55.343181,1.650839e-04,1.295049e-05,3.205301e-03,4.414119e-05,0.013689,0.000047,7.0,175.0,50.0
20,WASP-36 b,WASP-36,1.327,0.021,131.580390,-8.026986,0.13677,0.00056,5959.0,134.0,...,39.159947,1.125718e-05,1.645525e-06,2.011911e-03,5.690786e-05,0.018715,0.000153,4.0,71.0,19.0
28,HAT-P-1 b,HAT-P-1,1.319,0.019,344.445363,38.674919,0.11802,0.00018,5980.0,49.0,...,11.941089,5.118160e-07,4.317266e-08,7.394700e-04,1.298549e-05,0.013927,0.000041,3.0,18.0,427.0
248,XO-5 b,XO-5,1.140,0.030,116.716387,39.094470,0.10390,0.00070,5430.0,70.0,...,15.309169,1.662786e-07,2.035843e-08,4.849511e-04,1.309665e-05,0.010791,0.000145,2.0,13.0,26.0


In [ ]:
# load from csv
candidates = read_csv('subtargets.csv')

candidates.keys()


Index(['pl_name', 'hostname', 'pl_radj', 'pl_radjerr1', 'ra', 'dec',
       'pl_ratror', 'pl_ratrorerr1', 'st_teff', 'st_tefferr1', 'st_met',
       'st_logg', 'st_loggerr1', 'st_mass', 'st_rad', 'st_raderr1', 'Jmag',
       'Tmag', 'TIC', 'tmid', 'tmid_err', 'emid', 'emid_err', 'per', 'per_err',
       'ars', 'ars_err', 'inc', 'inc_err', 'ecc', 'ecc_err', 'omega',
       'omega_err', 'Teq', 'Teq_err', 'edepth_vis', 'edepth_vis_err',
       'edepth_ir', 'edepth_ir_err', 'depth', 'depth_err', 'n_sectors', 'n_lc',
       'n_rv'],
      dtype='object')

In [ ]:
from bokeh.models import ColumnDataSource, DataTable, TableColumn
from bokeh.layouts import column
from bokeh.io import show
from bokeh.io import output_file


keep_keys = ['pl_name', 'TIC', 'pl_radj', 'ra', 'dec',
       'pl_ratror', 'st_teff', 'st_met',
       'st_logg','st_mass', 'st_rad', 'Jmag',
       'Tmag', 'tmid', 'tmid_err', 'emid', 'emid_err', 'per', 
       'ars', 'inc', 'ecc', 'omega', 'Teq', 'edepth_vis', 
       'edepth_ir', 'depth', 'n_sectors', 'n_lc', 'n_rv']

candidates = candidates[keep_keys]

# round Teq to nearest integer
candidates['Teq'] = candidates['Teq'].round(1)

# convert e_depth to ppm
candidates['edepth_vis'] = candidates['edepth_vis'] * 1e6
candidates['edepth_vis'] = candidates['edepth_vis'].round(1)
candidates['edepth_ir'] = candidates['edepth_ir'] * 1e6
candidates['edepth_ir'] = candidates['edepth_ir'].round(1)
candidates['depth'] = candidates['depth'] * 1e6
candidates['depth'] = candidates['depth'].round(1)

# round to nearest integer
candidates['st_teff'] = candidates['st_teff'].round(0)
candidates['st_met'] = candidates['st_met'].round(3)
candidates['st_logg'] = candidates['st_logg'].round(3)

# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(candidates)

# Define the columns for the table
columns = [TableColumn(field=col, title=col) for col in candidates.columns]

# Create the DataTable
data_table = DataTable(source=source, columns=columns, width=1920, sortable=True,
                       sizing_mode='stretch_both', autosize_mode='fit_viewport')

# save html
output_file('candidates.html')

# Display the table
show(column(data_table))

In [ ]:
# filter for targets with TESS data
mask = candidates.n_lc > 0

candidates = candidates[mask]


# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(candidates)

# Define the columns for the table
columns = [TableColumn(field=col, title=col) for col in candidates.columns]

# Create the DataTable
data_table = DataTable(source=source, columns=columns, width=1920, sortable=True,
                       sizing_mode='stretch_both', autosize_mode='fit_viewport')

# save html
output_file('candidates_with_tess.html')

# Display the table
show(column(data_table))


In [ ]:
# save html
from bokeh.io import output_file

# filter for targets with TESS data
mask = (candidates.n_lc > 0) & (candidates.n_rv > 0)

candidates = candidates[mask]


# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(candidates)

# Define the columns for the table
columns = [TableColumn(field=col, title=col) for col in candidates.columns]

# Create the DataTable
data_table = DataTable(source=source, columns=columns, width=1920, sortable=True,
                       sizing_mode='stretch_both', autosize_mode='fit_viewport')

# save html
output_file('candidates_with_tess_rv.html')

# Display the table
show(column(data_table))